<a href="https://colab.research.google.com/github/jasonwong-lab/HKU-Practical-Bioinformatics/blob/main/NGS_sequence_alignment_command_line.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### BBMS3004 2026 - Targeted sequencing in acute myeloid leukaemia

*by Alvin Ip*

This practical aims to demonstrate how next-generation sequencing data is processed to yield clinically useful information. It is a highly condensed version of BIOF3002 Clinical Genomics and Bioinformatics that Prof Jason Wong and I co-coordinated for bioinformatics students.


## *** Package installation and downloads for workshop (~ 10 minutes)

1.   conda (for simple installation of packages)
2.   FastQC (for reads quality check)
3.   Trimmomatic (for trimming low-quality bases and adapters)
4.   bwa (tools for sequence alignment)
5.   samtools (tools for processing sam & bam files)  

**IMPORTANT：Every time you connect to Google Colab, you have to perform these set up steps again.**

In [ ]:
# Install terminal for Google Colab
!pip install colab-xterm
%load_ext colabxterm
%xterm

In [ ]:
# Set working pathway to your own Google drive, so that the data files are saved after closing Colab (~ 1 min)
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Install igv-notebook (<1 min)
!pip install igv-notebook

In [ ]:
# Install conda (~ 1 min). There will be a message saying that the session has crashed, but don't worry about this. This is due to the session restarting following conda installation.
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# Install fastqc (~ 2 mins)
!conda install -c bioconda fastqc

In [ ]:
# Install bwa (~ 1 min)
!conda install -c bioconda bwa

In [ ]:
# Install samtools (~1 min)
#!conda install -c bioconda samtools

In [ ]:
# Install samtools, bcftools, htslib, bedtools, bamtools (~ 2 mins)
!conda install -y -c conda-forge -c bioconda \
  samtools=1.20 bcftools=1.20 htslib=1.20 bedtools=2.31.1 bamtools

In [ ]:
# Check installation ran correctly
!samtools --version | head -1

In [ ]:
# Install GATK in a new environment (to specify the exapct openjdk and python version) ~1.5 mins
!conda create -y -n gatkenv python=3.10
!conda config --env --set channel_priority strict
!conda install -y -n gatkenv -c conda-forge -c bioconda gatk4=4.6.2.0 openjdk=17

In [ ]:
# Check that gatk is installed properly
!conda run -n gatkenv gatk --version

In [ ]:
#Install Trimmomatic
#!apt-get install openjdk-8-jre-headless -qq  # Install Java
#!wget http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip  # Download Trimmomatic
#!unzip Trimmomatic-0.39.zip  # Unzip the downloaded file

## Set working directory

By default, the working directory will be My Drive/bbms3004

In [ ]:
import os
try:
  os.mkdir("/content/gdrive/My Drive/bbms3004")         # change this path if necessary
except FileExistsError:
  print("directory already exist. OK to continue")
os.chdir("/content/gdrive/My Drive/bbms3004")

## Download ready prepared files for analysis.

In [ ]:
# Download data files from Google Drive if not already present (~ 3 min)
# Double check that we are in the right directory (~ 30s)
import os
os.chdir("/content/gdrive/MyDrive/bbms3004")                     # change this path if necessary

import os
if os.path.isfile("/content/gdrive/MyDrive/bbms3004/bbms3004_toolkit/17H0510082_1.fastq.gz"):    # check if the file exist
  print("Data files already exist, OK to continue.")
else:
  !pip install gdown
  #!gdown -O bbms3004_toolkit.zip https://drive.google.com/uc?id=1aRJVznjy5WLQ5Dc0DT9c6NiXw64HdoKr # download if file not exist
  !gdown -O bbms3004_toolkit.zip https://drive.google.com/uc?id=19Ql6GbWow2jj77UvQwCBMevnOmRs6S4I # download if file not exist
  # unzip toolkit files
  !unzip -o bbms3004_toolkit.zip
  # remove the downloaded zip file after extraction
  !rm bbms3004_toolkit.zip

!ls -l ./bbms3004_toolkit/

## NGS_alignment command line

1.1 Quality control

1.2. Sequence alignment using Burrows–Wheeler Aligner (BWA)

1.3. Viewing SAM files

1.4. Align the WGS paired-end file

In [ ]:
# Check what datasets we have downloaded
%cd /content/gdrive/MyDrive/bbms3004/bbms3004_toolkit
!ls -l

%cd data
!ls -l

In [ ]:
# To look at the ChIP-seq fastq file type (head -n 12 to print first 12 lines):
!zcat 17H0510082_1.fastq.gz | head -n 12

In [ ]:
# How many reads are there in the file? Type your code below:


### Quality control

In [ ]:
## Run FastQC
!fastqc 17H0510082_1.fastq.gz
!fastqc 17H0510082_2.fastq.gz

In [ ]:
# Check the html file output by FastQC
!ls

# Download the html file and check it on your local browser
from google.colab import files
files.download('17H0510082_1_fastqc.html')

You can also download the file **17H0510082_1_fastqc.html** from "gdrive/MyDrive/bbms3004/Datasets" on the left side.

In [ ]:
# Perform Trimmomatic
!java -jar ../Trimmomatic-0.39/trimmomatic-0.39.jar PE -phred33 \
    17H0510082_1.fastq.gz 17H0510082_2.fastq.gz \
    17H0510082_1_trimmed_paired.fq.gz 17H0510082_1_trimmed_unpaired.fq.gz 17H0510082_2_trimmed_paired.fq.gz 17H0510082_2_trimmed_unpaired.fq.gz \
    ILLUMINACLIP:../Trimmomatic-0.39/adapters/TruSeq3-PE-2.fa:2:30:10 LEADING:10 TRAILING:10 SLIDINGWINDOW:4:15 MINLEN:40


### Burrows–Wheeler Aligner

In [ ]:
# Look at the options for bwa and bwa mem
!bwa

In [ ]:
!bwa mem

In [ ]:
# Let's take a look at the database files
%cd /content/gdrive/My Drive/bbms3004/bbms3004_toolkit/Ref

!ls -l

In [ ]:
# Get ready to run BWA: First go into the Datasets directory
%cd /content/gdrive/My Drive/bbms3004/bbms3004_toolkit/data

In [ ]:
# Do sequence alignment with the default options
#!bwa mem ../bbms3004_toolkit/chr2.fa ./ChIP-seq_H3K27ac_example.fq.gz > ./BAM/ChIP-seq_H3K27ac_example.sam
!bwa mem -t 4 -M ../Ref/ucsc.hg19.subset.fasta 17H0510082_1_trimmed_paired.fq.gz 17H0510082_2_trimmed_paired.fq.gz > 17H0510082.sam

In [ ]:
# Check the result
!head -n 20 17H0510082.sam

### Working with SAM files

In [ ]:
# Check the samtools command
!samtools

In [ ]:
# Sort the SAM file and compress SAM to BAM
!conda run -n gatkenv gatk SortSam \
    -I 17H0510082.sam \
    -O 17H0510082_sorted.bam \
    --SORT_ORDER coordinate

In [ ]:
# Add read groups (RG), tags for downstream tools to run properly
!conda run -n gatkenv gatk AddOrReplaceReadGroups \
    -I 17H0510082_sorted.bam \
    -O 17H0510082_RG.bam \
    --RGID SPACE --RGLB panel --RGPL ILLUMINA --RGPU unit1 --RGSM 17H0510082 --CREATE_INDEX true

In [ ]:
# Perform MarkDuplicates to mark duplicate reads
!conda run -n gatkenv gatk MarkDuplicates \
    -I 17H0510082_RG.bam \
    -O 17H0510082_MD.bam \
    --METRICS_FILE 17H0510082_MD.stats \
    --CREATE_INDEX true

In [ ]:
# Perform BaseRecalibrator to recalibrate base quality scores
!conda run -n gatkenv gatk BaseRecalibrator \
    -I 17H0510082_MD.bam \
    -R ../Ref/ucsc.hg19.subset.fasta \
    --known-sites../Ref/dbsnp_138.hg19.vcf \
    --known-sites ../Ref/Mills_and_1000G_gold_standard.indels.hg19.vcf \
    -L ../Ref/myeloid-targets.interval_list \
    -ip 50 \
    -O 17H0510082_recal_data.table

# Apply the recalibration to the sequence data
!conda run -n gatkenv gatk ApplyBQSR \
    -R ../Ref/ucsc.hg19.subset.fasta \
    -I 17H0510082_MD.bam \
    --bqsr-recal-file 17H0510082_recal_data.table \
    -O 17H0510082_BR.bam

In [ ]:
# Collect QC metrics
!conda run -n gatkenv gatk CollectMultipleMetrics \
    -I 17H0510082_BR.bam \
    -R ../Ref/ucsc.hg19.subset.fasta \
    -O 17H0510082_QC_metrics

# Collect read counts
!conda run -n gatkenv gatk CollectReadCounts \
    -I 17H0510082_BR.bam \
    -L ../Ref/myeloid-targets.interval_list \
    --interval-merging-rule OVERLAPPING_ONLY \
    --format TSV \
    -O 17H0510082_read_counts.table

# Collect HS metrics
!conda run -n gatkenv gatk CollectHsMetrics \
    -I 17H0510082_BR.bam \
    -O 17H0510082_hs_metrics.txt \
    -R ../Ref/ucsc.hg19.subset.fasta \
    -BAIT_INTERVALS ../Ref/myeloid-probe-coords.interval_list \
    -TARGET_INTERVALS ../Ref/myeloid-targets.interval_list

In [ ]:
# Perform variant calling with Mutect2
!conda run -n gatkenv gatk Mutect2 \
    -R ../Ref/ucsc.hg19.subset.fasta \
    -I 17H0510082_BR.bam \
    -L ../Ref/myeloid-targets.interval_list \
    --germline-resource ../Ref/af-only-gnomad.myeloid.bedtools.vcf.gz \
    --f1r2-tar-gz f1r2.tar.gz \
    -O 17H0510082_unfiltered.vcf.gz

!conda run -n gatkenv gatk LearnReadOrientationModel \
    -I f1r2.tar.gz \
    -O read-orientation-model.tar.gz

!conda run -n gatkenv gatk GetPileupSummaries \
    -I 17H0510082_BR.bam \
    -V ../Ref/small_exac_common_myeloid.vcf.gz \
    -L ../Ref/small_exac_common_myeloid.vcf.gz  \
    -O getpileupsummaries.table

!conda run -n gatkenv gatk CalculateContamination \
    -I getpileupsummaries.table \
    -O contamination.table

!conda run -n gatkenv gatk FilterMutectCalls \
    -V 17H0510082_unfiltered.vcf.gz \
    -R ../Ref/ucsc.hg19.subset.fasta \
    --contamination-table contamination.table \
    --ob-priors read-orientation-model.tar.gz \
    -O 17H0510082_filtered.vcf.gz

In [ ]:
# Use bcftools to filter variants with variant allele frequencies less than 0.05
!bcftools filter -i " FORMAT/AF " 17H0510082_filtered.vcf.gz -o 17H0510082_filtered_AF05.vcf

In [ ]:
# Perform variant annotation by ANNOVAR
!perl ../annovar/table_annovar.pl 17H0510082_filtered_AF05.vcf \
    -buildver hg19 \
    -out 17H0510082_filtered_annotate \
    -remove \
    -protocol refGene,cosmic86,clinvar_20170905,exac03nontcga,gnomad_exome \
    -operation g,f,f,f,f \
    -nastring . \
    -vcfinput